# Timing statistics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('ggplot')

In [ ]:
table = pd.read_json("recontiming128-1024_20210208.json", orient="index")
table.head()

In [ ]:
table['timing']['2021-02-08 13:50:06']

In [ ]:
table['total']=np.nan
table['Time/slice']=np.nan
table['size']=np.nan
table['Total GUPS']=np.nan
table['GUPS/slice']=np.nan
table['FullLogNorm']=np.nan
table['MorphSpotClean']=np.nan
table['WaveletRingClean']=np.nan
table['Multi projection BP parallel']=np.nan


for ind in table.index: 
    table['total'][ind] = table['timing'][ind]['total'] 
    datadict = table['data'][ind] 
    sum=1
    for key in datadict :
        sum = sum * datadict[key]
        
    timingdict = table['timing'][ind]
    for key in timingdict :
        table[key][ind]=timingdict[key]

        

    table['size'][ind]       = datadict['sizeu']  
    table['Time/slice'][ind] = table['total'][ind]/datadict['sizeu'] 
    table['Total GUPS'][ind] = sum/table['total'][ind]/1e9
    table['GUPS/slice'][ind] = table['Total GUPS'][ind]/datadict['sizez']**2
        

In [ ]:
table.head()

In [ ]:
sizes = []
avgTotalTime = []
stdTotalTime = []
avgSliceTime = []
stdSliceTime = []
avgGUPS = []
avgNorm = []
avgSpot = []
avgRing = []
avgBP   = []
avgOther = []

scale = 1000
for s in table['size'].unique() :
    sizes.append(int(s))
    avgTotalTime.append(table[table['size']==s]['total'].mean()/scale)
    stdTotalTime.append(table[table['size']==s]['total'].std()/scale)
    avgSliceTime.append(table[table['size']==s]['Time/slice'].mean()/scale)
    stdSliceTime.append(table[table['size']==s]['Time/slice'].std()/scale)
    avgGUPS.append(table[table['size']==s]['Total GUPS'].mean())
    avgNorm.append(table[table['size']==s]['FullLogNorm'].mean()/scale)
    avgSpot.append(table[table['size']==s]['MorphSpotClean'].mean()/scale)
    avgRing.append(table[table['size']==s]['WaveletRingClean'].mean()/scale)
    avgBP.append(table[table['size']==s]['Multi projection BP parallel'].mean()/scale)
    avgOther.append(avgTotalTime[-1]
                    -avgNorm[-1]
                    -avgSpot[-1]
                    -avgRing[-1]
                    -avgBP[-1])

data = {'size': sizes, 
        'Total time': avgTotalTime, 
        'std Total time': stdTotalTime, 
        'Time/slice': avgSliceTime, 
        'std Time/slice': stdSliceTime,
        'GUPS': avgGUPS,
        'Normalization': avgNorm,
        'Spot cleaning': avgSpot,
        'Ring cleaning': avgRing,
        'Back projection': avgBP,
        'Others': avgOther}

timings = pd.DataFrame.from_dict(data).sort_values('size')

In [ ]:
timings

In [ ]:
plt.figure(figsize=(5,4))
plt.plot(timings['size'],timings['Total time'])
plt.plot(timings['size'],timings['Total time'],'o')
plt.xlabel('Data size')
plt.ylabel('time (s)')
plt.title('Total reconstruction time');
plt.tight_layout()
plt.savefig('totaltime.svg')
plt.savefig('totaltime.png',dpi=300)

In [ ]:
plt.figure(figsize=(5,4))
plt.plot(timings['size'],timings['Time/slice'])
plt.plot(timings['size'],timings['Time/slice'],'o')
plt.xlabel('Data size')
plt.ylabel('time (s)')
plt.title('Reconstruction time per slice');
plt.tight_layout()
plt.savefig('timeperslice.svg')
plt.savefig('timeperslice.png',dpi=300)

In [ ]:
plt.figure(figsize=(5,4))
plt.plot(timings['size'],timings['Time/slice'])
plt.plot(timings['size'],timings['Time/slice'],'o')
plt.xlabel('Data size')
plt.ylabel('time (s)')
plt.title('Reconstruction time per slice');
plt.tight_layout()
plt.savefig('timeperslice.svg')
plt.savefig('timeperslice.png',dpi=300)

In [ ]:
plt.figure(figsize=(5,4))
plt.plot(timings['size'],timings['GUPS'])
plt.plot(timings['size'],timings['GUPS'],'o')
plt.xlabel('Data size')
plt.ylabel('GUPS')
plt.title('Update rate');
plt.tight_layout()
plt.savefig('GUPS.svg')
plt.savefig('GUPS.png',dpi=300)

In [ ]:
plt.figure(figsize=(5,5))
timings.plot('size',['Normalization','Spot cleaning','Ring cleaning','Back projection','Others'],style='.-')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(5,5))
timings.plot.bar('size',['Normalization','Spot cleaning','Ring cleaning','Back projection','Others'],stacked=True)
plt.xlabel('Data size')
plt.ylabel('Time (s)')
plt.tight_layout()
plt.savefig('timepertask.svg')
plt.savefig('timepertask.png',dpi=300)